# Using the ACCESS-NRI Intake catalog as an alternative to the COSIMA Cookbook

This notebook demonstrates how to use the ACCESS-NRI Intake catalog, and demonstrates key similarities to and differences from the cosima cookbook

This notebook is concise version of the longer [ACCESS-NRI Intake catalog documentation](https://access-nri-intake-catalog.readthedocs.io/) and related [COSIMA training workshop](https://github.com/ACCESS-Hive/cosima-training-workshop-2023/blob/main/Intake.ipynb). Users are encouraged to refer to these for more detail and demonstrations. At the time of writing (Oct 2023), the ACCESS-NRI Intake Catalog is under testing and feedback from users is requested.

Requirements: The `conda/analysis3` (tested on `analysis3-23.07`) module from `/g/data/hh5/public/modules`. 

# Too Long; Didn't Read:

The most commonly used method from the cosima cookbook is `querying.getvar`, e.g.:

```python
import cosima_cookbook as cc

session = cc.database.create_session()

da = cc.querying.getvar(
    expt="expt0", 
    variable="var0", 
    session=session, 
    frequency="1 monthly"
)
```

Using the ACCESS-NRI Intake catalog, the same data can be obtained with:

```python
import intake

catalog = intake.cat.access_nri

ds = catalog["expt0"].search(
    variable="var0", 
    frequency="1mon"
).to_dask()

da=ds[var0]
```

# Start a dask Client

This is not specific to using the ACCESS-NRI Intake catalog. You would do the same thing using the COSIMA cookbook.

In [1]:
from dask.distributed import Client

client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 46.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43707,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 46.00 GiB
Comm: tcp://127.0.0.1:43457,Total threads: 1
Dashboard: /proxy/41043/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:45893,


# Opening and searching the catalog

To use the ACCESS-NRI Intake catalog, we need to import `intake`

In [2]:
import intake
import cosima_cookbook as cc # We'll also load the cookbook for comparisons later

We can open the catalog as follows. This is somewhat similar to starting a database session with the COSIMA cookbook. The returned object `catalog` is an instance of the ACCESS-NRI Intake catalog that we can use to find and load data.

In [3]:
catalog = intake.cat.access_nri

Printing the `catalog` object will return a dataframe of experiments that you can browse:

In [4]:
catalog

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{seaIce, ocean}","{fx, 3hr, 1mon, 1day, 3mon}","{dxu, alidf_ai_m, total_ocean_evap, rhoave, v, pe_tot, total_ocean_hflux_evap, vicen_m, HTN, tx_trans_submeso, total_ocean_salt, pot_temp, dxt, geolat_c, tau_x, flatn_ai_m, swflx, tmask, sfc_salt_..."
01deg_jra55v140_iaf,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1day, fx, 1mon}","{dxu, alidf_ai_m, total_ocean_evap, rhoave, v, pe_tot, total_ocean_hflux_evap, mld_max, vicen_m, HTN, daidtd_m, total_ocean_salt, pot_temp, dxt, geolat_c, tau_x, flatn_ai_m, swflx, tmask, sfc_salt..."
01deg_jra55v140_iaf_cycle2,{ACCESS-OM2},{Cycle 2 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1day, fx, 1mon}","{dxu, alidf_ai_m, total_ocean_evap, rhoave, v, surface_pot_temp, pe_tot, total_ocean_hflux_evap, surface_temp_max, mld_max, vicen_m, HTN, daidtd_m, total_ocean_salt, pot_temp, dxt, surface_pot_tem..."
01deg_jra55v140_iaf_cycle3,{ACCESS-OM2},{Cycle 3 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1day, fx, 1mon}","{dxu, alidf_ai_m, total_ocean_evap, rhoave, v, surface_pot_temp, pe_tot, total_ocean_hflux_evap, mld_max, vicen_m, HTN, daidtd_m, total_ocean_salt, pot_temp, dxt, surface_pot_temp_max, geolat_c, t..."
01deg_jra55v140_iaf_cycle4,{ACCESS-OM2},{Cycle 4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},"{seaIce, ocean}","{6hr, fx, 3hr, 1mon, 1day}","{dxu, alidf_ai_m, no3_yflux_adv, alk, pe_tot, total_ocean_hflux_evap, src03, det_xflux_adv, tau_x, flatn_ai_m, npp3d, daidtt, phy_intmld, vvel_h, divu_m, adic_intmld, zoo, ht, src05, total_ocean_h..."
01deg_jra55v140_iaf_cycle4_jra55v150_extension,{ACCESS-OM2},{Extensions of cycle 4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 interannual forcing},"{seaIce, ocean}","{1day, fx, 1mon, subhr}","{dxu, alidf_ai_m, pprod_gross_int100, total_ocean_evap, fe_zflux_adv, rhoave, no3_yflux_adv, surface_no3, surface_pot_temp, v, det_int100, stf07, pe_tot, total_ocean_hflux_evap, src01, alk, mld_ma..."
01deg_jra55v150_iaf_cycle1,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0 OMIP2 interannual forcing},"{seaIce, ocean}","{1day, fx, 1mon}","{dxu, v, surface_pot_temp, mld_max, dxt, geolat_c, tau_x, swflx, sfc_salt_flux_ice, evap_heat, hi_m, eta_nonbouss, fprec, ht, sfc_salt_flux_restore, river, TLON, age_global, area_u, aice, kmt, dra..."
025deg_jra55_iaf_omip2_cycle1,{ACCESS-OM2},{Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{seaIce, ocean}","{1yr, 1day, fx, 1mon}","{dxu, alidf_ai_m, total_ocean_evap, rhoave, aredi, v, aiso_bih, pbot_adjust, pe_tot, neutral_gm_salt, total_ocean_hflux_evap, sfc_salt_flux_runoff, mld_max, vicen_m, HTN, daidtd_m, albice_m, tx_tr..."
025deg_jra55_iaf_omip2_cycle2,{ACCESS-OM2},{Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{seaIce, ocean}","{1yr, 1day, fx, 1mon}","{dxu, alidf_ai_m, total_ocean_evap, rhoave, aredi, v, aiso_bih, pbot_adjust, pe_tot, neutral_gm_salt, total_ocean_hflux_evap, sfc_salt_flux_runoff, mld_max, vicen_m, HTN, daidtd_m, albice_m, tx_tr..."


You can also search based on the columns in this dataframe to find experiments that are relevant to you. For example, you might be interested in all ACCESS-OM2 experiments that have the variable `"surface_salt"` at daily frequency. There are 6 such experiments currently available through the catalog:

In [5]:
catalog.search(model="ACCESS-OM2", variable="surface_salt", frequency="1day")

,model,description,realm,frequency,variable
name,,,,,
01deg_jra55v13_ryf9091,{ACCESS-OM2},{0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},{ocean},{1day},{surface_salt}
01deg_jra55v140_iaf,{ACCESS-OM2},{Cycle 1 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},{ocean},{1day},{surface_salt}
01deg_jra55v140_iaf_cycle2,{ACCESS-OM2},{Cycle 2 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},{ocean},{1day},{surface_salt}
01deg_jra55v140_iaf_cycle3,{ACCESS-OM2},{Cycle 3 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},{ocean},{1day},{surface_salt}
01deg_jra55v140_iaf_cycle4,{ACCESS-OM2},{Cycle 4 of 0.1 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0 OMIP2 interannual forcing},{ocean},{1day},{surface_salt}
01deg_jra55v140_iaf_cycle4_jra55v150_extension,{ACCESS-OM2},{Extensions of cycle 4 of 0.1 degree ACCESS-OM2 + WOMBAT BGC global model configuration with JRA55-do v1.5.0 and v1.5.0.1 interannual forcing},{ocean},{1day},{surface_salt}


In this way, the catalog provides similar functionality to the COSIMA cookbook Database Explorer tool.

# Opening data

There are [multiple ways](https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html#loading-intake-sources) to open data from the experiments in `catalog`. Here we'll demonstrate how to do this when you know the name of the experiment you are interested in, since this typical for COSIMA users.

By far the most commonly used COSIMA cookbook function is `querying.getvar`. For example, we might use it to open monthly data for the `surface_salt` variable in the `01deg_jra55v13_ryf9091` experiment as follows:

In [6]:
experiment = "01deg_jra55v13_ryf9091"
variable = "surface_salt"

In [7]:
session = cc.database.create_session()

data_cc = cc.querying.getvar(
    expt=experiment,
    variable=variable,
    session=session,
    frequency="1 monthly",
)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3

In [8]:
data_cc

<xarray.DataArray 'surface_salt' (time: 3360, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<concatenate, shape=(3360, 2700, 3600), dtype=float32, chunksize=(1, 675, 900), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * time      (time) object 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
Attributes: (12/13)
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_surface_salinity
    ...             ...
    contact:        Andy Hogg
    email:          andy.hogg@anu.edu.au
    created:        2020-06-11
    description:    0.1 degree ACCESS-OM2 global model configuration with JRA...
    notes:          Additional daily outputs saved from 1 Jan 1950 to 31 Dec ...
    url:            https://github.com/COSIMA/01deg_jra55_ryf/tree/01deg_jra5...

We can open the same data using our `catalog` object as follows:

In [9]:
data_ic = catalog[experiment].search(
    variable=variable, 
    frequency="1mon"
).to_dask()

In [10]:
data_ic["surface_salt"]

<xarray.DataArray 'surface_salt' (time: 3360, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<concatenate, shape=(3360, 2700, 3600), dtype=float32, chunksize=(1, 675, 900), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * time      (time) object 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_surface_salinity

# Some important differences

There are a few differences in functionality between the ACCESS-NRI Intake catalog and the COSIMA cookbook that users should be aware of.

## 1. The cookbook returns DataArrays, whereas the catalog returns Datasets

This is because with the catalog you can load multiple variables into a single dataset with a single call (when these variables are in the same file). E.g.:

In [11]:
data_ic_multivar = catalog[experiment].search(
    variable=["surface_salt", "surface_temp"], 
    frequency="1mon"
).to_dask()

In [12]:
data_ic_multivar

<xarray.Dataset>
Dimensions:       (time: 3360, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * time          (time) object 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
Data variables:
    surface_salt  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
Attributes:
    filename:                        ocean_month.nc
    title:                           ACCESS-OM2-01
    grid_type:                       mosaic
    grid_tile:                       1
    intake_esm_attrs:realm:          ocean
    intake_esm_attrs:frequency:      1mon
    intake_esm_attrs:filename:       ocean_month.nc
    intake_esm_attrs:file_id:        ocean_month
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          ocean_month.1mon

## 2. The catalog knows which files make up distinct datasets

The following fails with the cookbook because you will be trying to load data from different files with different temporal frequency:

In [13]:
data_ic_multifreq = cc.querying.getvar(
    expt=experiment,
    variable=variable,
    session=session,
)

QueryWarning: Your query returns files with differing frequencies: {'1 daily', '1 monthly'}. This could lead to unexpected behaviour! Disambiguate by passing frequency= to getvar, specifying the desired frequency.

Additionally, there are cases where the cookbook will [silently load and concatenate data on different grids](https://github.com/COSIMA/cosima-recipes/issues/229), because it does not know which files in the database should/shouldn't be concatenated together.

The catalog know which files make up distinct datasets and provides methods to open multiple datasets from a single query. For example, if we try to run the equivalent to the cell above using the catalog, we are told that there are multiple datasets and that we should open them with `to_dataset_dict()` rather than `to_dask()`:

In [14]:
data_ic_multifreq = catalog[experiment].search(variable=variable).to_dask()

ValueError: Expected exactly one dataset. Received 2 datasets. Please refine your search or use `.to_dataset_dict()`.

Doing so returns a dictionary containing Datasets of the variable at all the available frequencies (daily and monthly in this case)

In [15]:
data_ic_multifreq = catalog[experiment].search(variable=variable).to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id.frequency'


In [16]:
data_ic_multifreq

{'ocean_daily.1day': <xarray.Dataset>
 Dimensions:       (time: 81580, yt_ocean: 2700, xt_ocean: 3600)
 Coordinates:
   * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
   * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
   * time          (time) object 1956-04-01 12:00:00 ... 2179-12-31 12:00:00
 Data variables:
     surface_salt  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
 Attributes:
     filename:                        ocean_daily.nc
     title:                           ACCESS-OM2-01
     grid_type:                       mosaic
     grid_tile:                       1
     intake_esm_vars:                 ['surface_salt']
     intake_esm_attrs:realm:          ocean
     intake_esm_attrs:frequency:      1day
     intake_esm_attrs:filename:       ocean_daily.nc
     intake_esm_attrs:file_id:        ocean_daily
     intake_esm_attrs:_data_format_:  netcdf
     intake_esm_dataset_key

You can also open multiple datasets directly into an [xarray-datatree](https://xarray-datatree.readthedocs.io/en/latest/) by calling `to_datatree` rather than `to_dataset_dict` (in an upcoming release, it will be easier for users to control how the groups are structured in the datatree.). E.g.:

In [17]:
data_ic_datatree = catalog[experiment].search(variable=variable).to_datatree()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_id/frequency'


In [18]:
data_ic_datatree

DataTree('None', parent=None)
├── DataTree('ocean_daily')
│   └── DataTree('1day')
│           Dimensions:       (time: 81580, yt_ocean: 2700, xt_ocean: 3600)
│           Coordinates:
│             * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
│             * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
│             * time          (time) object 1956-04-01 12:00:00 ... 2179-12-31 12:00:00
│           Data variables:
│               surface_salt  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
│           Attributes:
│               filename:                        ocean_daily.nc
│               title:                           ACCESS-OM2-01
│               grid_type:                       mosaic
│               grid_tile:                       1
│               intake_esm_vars:                 ['surface_salt']
│               intake_esm_attrs:realm:          ocean
│               intake_esm_attrs:frequency:      1day
│               intake_esm_attrs:filename:       ocean_daily.nc
│               intake_esm_attrs:file_id:        ocean_daily
│               intake_esm_attrs:_data_format_:  netcdf
│               intake_esm_dataset_key:          ocean_daily/1day
└── DataTree('ocean_month')
    └── DataTree('1mon')
            Dimensions:       (time: 3360, yt_ocean: 2700, xt_ocean: 3600)
            Coordinates:
              * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
              * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
              * time          (time) object 1900-01-16 12:00:00 ... 2179-12-16 12:00:00
            Data variables:
                surface_salt  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 675, 900), meta=np.ndarray>
            Attributes:
                filename:                        ocean_month.nc
                title:                           ACCESS-OM2-01
                grid_type:                       mosaic
                grid_tile:                       1
                intake_esm_vars:                 ['surface_salt']
                intake_esm_attrs:realm:          ocean
                intake_esm_attrs:frequency:      1mon
                intake_esm_attrs:filename:       ocean_month.nc
                intake_esm_attrs:file_id:        ocean_month
                intake_esm_attrs:_data_format_:  netcdf
                intake_esm_dataset_key:          ocean_month/1mon

## 3. The frequency vocabulary is different

With the cookbook `querying.getvar` function, the provided frequency must be one of (where `<int>` is an integer):

```python
"static"
"<int> hourly"
"<int> daily"
"<int> monthly"
"<int> yearly"
```  

In the catalog, frequency follows a standard vocabulary that is very similar to CMIP6:

```python
"fx" # fixed
"subhr" # subhourly
"<int>hr" # hourly
"<int>day" # daily
"<int>mon" # monthly
"<int>yr" # yearly
"<int>dec" # decadal
```

## 4. Passing keyword arguments is different

With the cookbook `querying.getvar` function, additional keyword arguments to be passed to xarray's `open_mfdataset` can be passed directly to the `getvar` function. For example, one might specify the chunks and combining arguments using:

In [19]:
data_cc_kw = cc.querying.getvar(
    expt=experiment,
    variable=variable,
    session=session,
    frequency="1 monthly",
    chunks={"xt_ocean": -1, "yt_ocean": -1},
    compat="override",
    data_vars="minimal",
    coords="minimal"
)

With the catalog, keyword argments for xarray's `open_dataset` and `combine_by_coords` functions are passed separately to `to_dask` (or `to_dataset_dict`). For example, to do the same as the previous cell:

In [20]:
xarray_open_kwargs=dict(
    chunks={"xt_ocean": -1, "yt_ocean": -1}
)
xarray_combine_by_coords_kwargs=dict(
    compat="override",
    data_vars="minimal",
    coords="minimal"
)

data_ic_kw = catalog[experiment].search(
    variable=variable, 
    frequency="1mon"
).to_dask(
    xarray_open_kwargs=xarray_open_kwargs,
    xarray_combine_by_coords_kwargs=xarray_combine_by_coords_kwargs,
)

## 5. You cannot search by start and end date using the catalog

With the cookbook, it's common to specify a time *range* in the `getvar` query. E.g.:

In [21]:
start_time = "2000-01-01"
end_time = "2180-01-01"

data_cc_times = cc.querying.getvar(
    expt=experiment,
    variable=variable,
    session=session,
    frequency="1 monthly",
    start_time=start_time,
    end_time=end_time,
)

data_cc_times = data_cc_times.sel(time=slice(start_time, end_time))

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/xarray/core/dataset.py:278: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3

It's not possible to query on a time range with the Intake catalog. This is mostly because the opening of datasets is a trivially parallelized task that is done [lazily](https://docs.xarray.dev/en/stable/user-guide/dask.html#parallel-computing-with-dask) anyway so opening all files and reducing the times using xarray's `sel` methods shouldn't add too much overhead. In most cases where the overhead of opening the files seems large, this can be reduced through sensible choices of keyword arguments provided to `open_dataset` and `combine_by_coords` - see the xarray documentation on [Reading multi-file datasets](https://docs.xarray.dev/en/stable/user-guide/io.html#reading-multi-file-datasets) for details.

That is, with the catalog you'd just do (which takes a few seconds longer):

In [22]:
data_ic = catalog[experiment].search(
    variable=variable, 
    frequency="1mon"
).to_dask()

data_ic_times = data_ic.sel(time=slice(start_time, end_time))

# Tips, gotchas and workarounds

## 1. Speeding up opening your datasets

Try passing the following to your `to_dask` or `to_dataset_dict` call:

```python
xarray_combine_by_coords_kwargs=dict(
    compat="override",
    data_vars="minimal",
    coords="minimal"
)
```

See the xarray documentation on [Reading multi-file datasets](https://docs.xarray.dev/en/stable/user-guide/io.html#reading-multi-file-datasets) for more details about these arguments.

## 2. Choosing chunksizes

Check out the [Chunking tutorial](https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/chunking.html) in the ACCESS-NRI Intake catalog documentation

## 3. Loading fixed frequency variables

Many COSIMA experiments include multiple repeated files containing the same fixed frequency data (e.g. grid information). Both the cookbook and the catalog will fail to concatenate these files since they contain no clear dimension to concatenate along. The workaround with the cookbook is to specify the argument `n=1` to tell the cookbook to return only data from the first file, e.g.

In [23]:
data_cc_fixed = cc.querying.getvar(
    expt=experiment,
    variable="area_t",
    session=session,
    n=1,
)

There is no equivalent `n` argument with the catalog, but you can further restrict your search to only return one file. For example, you could only return the file in the `output000` directory with:

In [24]:
data_ic_fixed = catalog[experiment].search(
    variable='area_t',
    path=".*output000.*"
).to_dask()

## 4. Determining what can be searched upon in an experiment

You can see what can be `search`ed on within an experiment with:

In [25]:
catalog[experiment].df.columns.tolist()

['path',
 'realm',
 'variable',
 'frequency',
 'start_date',
 'end_date',
 'variable_long_name',
 'variable_standard_name',
 'variable_cell_methods',
 'variable_units',
 'filename',
 'file_id']

It can also be helpful sometimes to look at the `catalog[experiment].df` object itself, which is a dataframe of all of the files in the experiment and their metadata

In [26]:
catalog[experiment].df.head()

,path,realm,variable,frequency,start_date,end_date,variable_long_name,variable_standard_name,variable_cell_methods,variable_units,filename,file_id
0,/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-01.nc,seaIce,"[time_bounds, TLON, TLAT, ULON, ULAT, NCAT, tmask, blkmask, tarea, uarea, dxt, dyt, dxu, dyu, HTN, HTE, ANGLE, ANGLET, hi_m, hs_m, Tsfc_m, aice_m, uvel_m, vvel_m, uatm_m, vatm_m, fswup_m, sst_m, s...",1mon,"1900-01-01, 00:00:00","1900-02-01, 00:00:00","[boundaries for time-averaging interval, T grid center longitude, T grid center latitude, U grid center longitude, U grid center latitude, category maximum thickness, ocean grid mask, ice grid blo...","[, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ]","[, , , , , , , , , , , , , , , , , , time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, tim...","[days since 1900-01-01 00:00:00, degrees_east, degrees_north, degrees_east, degrees_north, m, , , m^2, m^2, m, m, m, m, m, m, radians, radians, m, m, C, 1, m/s, m/s, m/s, m/s, W/m^2, C, ppt, m/s, ...",iceh.1900-01.nc,iceh_XXXX_XX
1,/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-02.nc,seaIce,"[time_bounds, TLON, TLAT, ULON, ULAT, NCAT, tmask, blkmask, tarea, uarea, dxt, dyt, dxu, dyu, HTN, HTE, ANGLE, ANGLET, hi_m, hs_m, Tsfc_m, aice_m, uvel_m, vvel_m, uatm_m, vatm_m, fswup_m, sst_m, s...",1mon,"1900-02-01, 00:00:00","1900-03-01, 00:00:00","[boundaries for time-averaging interval, T grid center longitude, T grid center latitude, U grid center longitude, U grid center latitude, category maximum thickness, ocean grid mask, ice grid blo...","[, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ]","[, , , , , , , , , , , , , , , , , , time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, tim...","[days since 1900-01-01 00:00:00, degrees_east, degrees_north, degrees_east, degrees_north, m, , , m^2, m^2, m, m, m, m, m, m, radians, radians, m, m, C, 1, m/s, m/s, m/s, m/s, W/m^2, C, ppt, m/s, ...",iceh.1900-02.nc,iceh_XXXX_XX
2,/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ice/OUTPUT/iceh.1900-03.nc,seaIce,"[time_bounds, TLON, TLAT, ULON, ULAT, NCAT, tmask, blkmask, tarea, uarea, dxt, dyt, dxu, dyu, HTN, HTE, ANGLE, ANGLET, hi_m, hs_m, Tsfc_m, aice_m, uvel_m, vvel_m, uatm_m, vatm_m, fswup_m, sst_m, s...",1mon,"1900-03-01, 00:00:00","1900-04-01, 00:00:00","[boundaries for time-averaging interval, T grid center longitude, T grid center latitude, U grid center longitude, U grid center latitude, category maximum thickness, ocean grid mask, ice grid blo...","[, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ]","[, , , , , , , , , , , , , , , , , , time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, time: mean, tim...","[days since 1900-01-01 00:00:00, degrees_east, degrees_north, degrees_east, degrees_north, m, , , m^2, m^2, m, m, m, m, m, m, radians, radians, m, m, C, 1, m/s, m/s, m/s, m/s, W/m^2, C, ppt, m/s, ...",iceh.1900-03.nc,iceh_XXXX_XX
3,/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/ocean.nc,ocean,"[temp, pot_temp, salt, age_global, u, v, wt, dzt, rho, pot_rho_0, tx_trans, ty_trans, ty_trans_submeso, tx_trans_rho, ty_trans_rho, ty_trans_nrho_submeso, temp_xflux_adv, temp_yflux_adv, average_T...",3mon,"1900-01-01, 00:00:00","1900-04-01, 00:00:00","[Conservative temperature, Potential temperature, Practical Salinity, Age (global), i-current, j-current, dia-surface velocity T-points, t-cell thickness, in situ density, potential density refere...

If you have any further questions after reading this notebook and the documentation linked from this notebook, please open an issue in the [ACCESS-NRI Intake catalog Github repo](https://github.com/ACCESS-NRI/access-nri-intake-catalog) or open topic on the [ACCESS-Hive forum](https://forum.access-hive.org.au/).

In [27]:
client.close()